# Flight booking Assistant

As a first step into AI agentization, I built a multi-modal agency helping the user to book a flight:
- chat function (openAI) accessible via Gradio
- apart from general chat, the assistant has access to 2 tools: get_ticket_price(city) to find the ticket price, and get_destinations() to show all possible destinations
- there are also 3 functions ("specialized agents"): one who creates a nice picture, one who talks back, and one who says something cool in the local language of the destination

The objective is just to show how the complex problem into smaller steps, with multiple LLMs carrying out specialized tasks. And how this all works together.

The term 'Agentic AI' and Agentization is an umbrella term that refers to a number of techniques, such as:

1. Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
2. The ability for LLMs to use Tools to give them additional capabilities
3. The 'Agent Environment' which allows Agents to collaborate
4. An LLM can act as the Planner, dividing bigger tasks into smaller ones for the specialists
5. The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory

This project covers 1 and 2. Later on we will see more.

### 0. Imports and initialization

In [108]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import time

# for voice
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

# artist
import base64
from io import BytesIO
from PIL import Image

In [2]:
# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


### 1. Tools

In [69]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

In [13]:
# Tool 1
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [7]:
get_ticket_price("London")

Tool get_ticket_price called for London


'$799'

In [15]:
# Tool 2
def get_destinations():
    print(f"Tool get_destinations called")
    return list(ticket_prices.keys())

In [16]:
get_destinations()

Tool get_destinations called


['london', 'paris', 'tokyo', 'berlin']

In [17]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [19]:
destination_function = {
    "name": "get_destinations",
    "description": "Get the list of all available destination cities. Call this whenever you need to know the potential destinations, for example when a customer asks what destinations are available, \
        or when a customer asks the price for a destination that is not available"
}

In [21]:
# And this is included in a list of tools:
tools = [{"type": "function", "function": price_function},
        {"type": "function", "function": destination_function}]

### 2. Agents

In [48]:
# voice agent
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx", #alloy
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

In [67]:
# artist
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vintage 1970s style", # vibrant pop-art
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

#image = artist("London")
#display(image)

In [94]:
# Local language greeting
def welcome(city):
    user_prompt = "The user has just booked a flight to "
    user_prompt += city
    user_prompt += ". Determine what is the first official language spoken in this city, and the country where it is. "
    user_prompt += "Welcome the user to the country. For instance say 'welcome to France' for city Paris. "
    user_prompt += "Next, advice two nice local foods that the user should try. For instance say: 'you should definitely try the local croissants and cheese'. "
    user_prompt += "Say all this in the local language"
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": "You are a travel assistant who can say something interesting on any city, in the local language."},
            {"role": "user", "content": user_prompt}
      ])
    
    return response.choices[0].message.content

In [99]:
talker(welcome("Paris"))

### 3. Agent framework and Gradio

In [20]:
# System message
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [112]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        if city != "world": # then the tool call was a price request
            #image = artist(city)
            talker(welcome(city))
            time.sleep(5)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    print(history)

    # Talker agent
    talker(reply)
    
    return history, image

In [110]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == 'get_ticket_price':
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city
        
    elif tool_call.function.name == 'get_destinations':
        destinations = get_destinations()
        response = {
            "role": "tool",
            "content": json.dumps(destinations),
            "tool_call_id": tool_call.id
        }
        return response, "world"

In [113]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'metadata': None, 'content': 'Hello, who are you', 'options': None}, {'role': 'assistant', 'content': 'I am a virtual assistant for FlightAI, here to help you with your airline inquiries.'}]


[{'role': 'user', 'metadata': None, 'content': 'Hello, who are you', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'I am a virtual assistant for FlightAI, here to help you with your airline inquiries.', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'Nice, I want to book a flight', 'options': None}, {'role': 'assistant', 'content': 'Sure! Where would you like to fly to?'}]


Tool get_destinations called
[{'role': 'user', 'metadata': None, 'content': 'Hello, who are you', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'I am a virtual assistant for FlightAI, here to help you with your airline inquiries.', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'Nice, I want to book a flight', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Sure! Where would you like to fly to?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'What are my options?', 'options': None}, {'role': 'assistant', 'content': 'You can fly to London, Paris, Tokyo, or Berlin.'}]


Tool get_ticket_price called for Berlin


[{'role': 'user', 'metadata': None, 'content': 'Hello, who are you', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'I am a virtual assistant for FlightAI, here to help you with your airline inquiries.', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'Nice, I want to book a flight', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Sure! Where would you like to fly to?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'What are my options?', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'You can fly to London, Paris, Tokyo, or Berlin.', 'options': None}, {'role': 'user', 'metadata': None, 'content': "What's the price to Berlin?", 'options': None}, {'role': 'assistant', 'content': 'The price to Berlin is $499.'}]


Tool get_ticket_price called for Paris


[{'role': 'user', 'metadata': None, 'content': 'Hello, who are you', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'I am a virtual assistant for FlightAI, here to help you with your airline inquiries.', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'Nice, I want to book a flight', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Sure! Where would you like to fly to?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'What are my options?', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'You can fly to London, Paris, Tokyo, or Berlin.', 'options': None}, {'role': 'user', 'metadata': None, 'content': "What's the price to Berlin?", 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'The price to Berlin is $499.', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'And to Paris?', 'options': None}, {'role': 'assistant', 'content': 'The price to Paris is $899.'}]